# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import copy

def nhl_correlation():
    
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean nhl df
    def clean_nhl_df(teams_df):
        
        nhl_df_cleaned = copy.copy(teams_df)

        # Select data from 2018 only
        nhl_df_cleaned = nhl_df_cleaned[teams_df['year']==2018]

        nhl_df_cleaned.drop(labels=[0, 9, 18, 26], inplace=True)

        # Eliminate * at the end of the team name
        def fix_team_name(row):
            pattern = '.+?\*'
            if bool(re.search(pattern, row['team'])):
                row['team'] = re.sub('\*', '', row['team']).strip()

            return row

        nhl_df_cleaned = nhl_df_cleaned.apply(lambda x: fix_team_name(x), axis='columns')

        # Get metropolitan area per team
        def get_metro_area(team):
            metropolitan_areas = {'New York City': ['New York Islanders', 'New York Rangers', 'New Jersey Devils'],
                                  'Los Angeles': ['Los Angeles Kings', 'Anaheim Ducks'],
                                  'San Francisco Bay Area': ['San Jose Sharks'],
                                  'Chicago': ['Chicago Blackhawks'],
                                  'Dallas–Fort Worth': ['Dallas Stars'],
                                  'Washington, D.C.': ['Washington Capitals'],
                                  'Philadelphia': ['Philadelphia Flyers'],
                                  'Boston': ['Boston Bruins'],
                                  'Minneapolis–Saint Paul': ['Minnesota Wild'],
                                  'Denver': ['Colorado Avalanche'],
                                  'Miami–Fort Lauderdale': ['Florida Panthers'],
                                  'Phoenix': ['Arizona Coyotes'],
                                  'Detroit': ['Detroit Red Wings'],
                                  'Toronto': ['Toronto Maple Leafs'],
                                  'Tampa Bay Area': ['Tampa Bay Lightning'],
                                  'Pittsburgh': ['Pittsburgh Penguins'],
                                  'St. Louis': ['St. Louis Blues'],
                                  'Nashville': ['Nashville Predators'],
                                  'Buffalo': ['Buffalo Sabres'],
                                  'Montreal': ['Montreal Canadiens'],
                                  'Vancouver': ['Vancouver Canucks'],
                                  'Columbus': ['Columbus Blue Jackets'],
                                  'Calgary': ['Calgary Flames'],
                                  'Ottawa': ['Ottawa Senators'],
                                  'Edmonton': ['Edmonton Oilers'],
                                  'Winnipeg': ['Winnipeg Jets'],
                                  'Las Vegas': ['Vegas Golden Knights'],
                                  'Raleigh': ['Carolina Hurricanes']}

            for m_area in metropolitan_areas:
                if team in metropolitan_areas[m_area]:
                    return m_area

        nhl_df_cleaned['Metropolitan area'] = nhl_df_cleaned['team'].apply(lambda x: get_metro_area(x))

        nhl_df_cleaned = nhl_df_cleaned.astype({'W': 'int64', 'L': 'int64'})

        return nhl_df_cleaned

    # Adjust cities df and set Metro Areas as index
    def clean_cities_df(cities_df):

        cities_clean = copy.copy(cities_df)

        # Rename Population column
        cities_clean.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

        # Change dtype for Population
        cities_clean = cities_clean.astype({'Population': 'int64'})

        cities_clean.set_index('Metropolitan area', inplace=True)

        return cities_clean['Population']
    
    # Clean nhl df
    teams_nhl_df = clean_nhl_df(nhl_df)
    
    # Group by Metropolitan area
    nhl_groups = teams_nhl_df.groupby('Metropolitan area').agg({'W': np.nansum, 'L': np.nansum})
    nhl_groups['Win/Loss ratio'] = nhl_groups['W']/(nhl_groups['W'] + nhl_groups['L'])
    
    # Adjust cities df and set Metro Areas as index
    metro_cities = clean_cities_df(cities)
    
    # Join nhl and cities dfs
    resulting_df = pd.merge(left=metro_cities, right=nhl_groups, how='inner', left_index=True, right_index=True)
    
    # raise NotImplementedError()
    
    population_by_region = resulting_df['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = resulting_df['Win/Loss ratio'].to_list() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import copy

def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def clean_nba_df(teams_df):
        
        nba_df_cleaned = copy.copy(teams_df)
        
        # Select data from 2018 only
        nba_df_cleaned = nba_df_cleaned[nba_df_cleaned['year']==2018]
        
        # Eliminate * (#) at the end
        def fix_team_name(row):
            
            parenth_pattern = r'.+?\s\(\d*\)$'
            if bool(re.search(parenth_pattern, row['team'])):
                row['team'] = re.sub('\s\(\d*\)$', '', row['team']).strip()
            
            asth_pattern = r'.+?\*$'
            if bool(re.search(asth_pattern, row['team'])):
                row['team'] = re.sub('\*$', '', row['team']).strip()
            
            return row
        
        nba_df_cleaned = nba_df_cleaned.apply(lambda x: fix_team_name(x), axis='columns')
        
        # Get metropolitan area per team
        def get_metro_area(team):
            metropolitan_areas = {'New York City': ['New York Knicks', 'Brooklyn Nets'],
                                  'Los Angeles': ['Los Angeles Lakers', 'Los Angeles Clippers'],
                                  'San Francisco Bay Area': ['Golden State Warriors'],
                                  'Chicago': ['Chicago Bulls'],
                                  'Dallas–Fort Worth': ['Dallas Mavericks'],
                                  'Washington, D.C.': ['Washington Wizards'],
                                  'Philadelphia': ['Philadelphia 76ers'],
                                  'Boston': ['Boston Celtics'],
                                  'Minneapolis–Saint Paul': ['Minnesota Timberwolves'],
                                  'Denver': ['Denver Nuggets'],
                                  'Miami–Fort Lauderdale': ['Miami Heat'],
                                  'Phoenix': ['Phoenix Suns'],
                                  'Detroit': ['Detroit Pistons'],
                                  'Toronto': ['Toronto Raptors'],
                                  'Houston': ['Houston Rockets'],
                                  'Atlanta': ['Atlanta Hawks'],
                                  'Cleveland': ['Cleveland Cavaliers'],
                                  'Charlotte': ['Charlotte Hornets'],
                                  'Indianapolis': ['Indiana Pacers'],
                                  'Milwaukee': ['Milwaukee Bucks'],
                                  'New Orleans': ['New Orleans Pelicans'],
                                  'Orlando': ['Orlando Magic'],
                                  'Portland': ['Portland Trail Blazers'],
                                  'Salt Lake City': ['Utah Jazz'],
                                  'San Antonio': ['San Antonio Spurs'],
                                  'Sacramento': ['Sacramento Kings'],
                                  'Oklahoma City': ['Oklahoma City Thunder'],
                                  'Memphis': ['Memphis Grizzlies']}
            
            for m_area in metropolitan_areas:
                if team in metropolitan_areas[m_area]:
                    return m_area
        
        nba_df_cleaned['Metropolitan area'] = nba_df_cleaned['team'].apply(lambda x: get_metro_area(x))
        
        nba_df_cleaned = nba_df_cleaned.astype({'W': 'int64', 'L': 'int64'})
        
        return nba_df_cleaned
    
    # Adjust cities df and set Metro areas as index
    def clean_cities_df(cities_df):

        cities_clean = copy.copy(cities_df)

        # Rename Population column
        cities_clean.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

        # Change dtype for Population
        cities_clean = cities_clean.astype({'Population': 'int64'})

        cities_clean.set_index('Metropolitan area', inplace=True)

        return cities_clean['Population']
    
    # Clean nba df
    teams_nba_df = clean_nba_df(nba_df)
    
    # Group by Metropolitan area
    nba_groups = teams_nba_df.groupby('Metropolitan area').agg({'W': np.nansum, 'L': np.nansum})
    nba_groups['Win/Loss ratio'] = nba_groups['W']/(nba_groups['W'] + nba_groups['L'])
    
    # Adjust cities df and set Metro Areas as index
    metro_cities = clean_cities_df(cities)
    
    # Join nba and cities dfs
    resulting_df = pd.merge(left=metro_cities, right=nba_groups, how='inner', left_index=True, right_index=True)
    
    # raise NotImplementedError()
    
    population_by_region = resulting_df['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = resulting_df['Win/Loss ratio'].to_list() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import copy

def mlb_correlation():
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def clean_mlb_df(teams_df):
        
        mlb_df_cleaned = copy.copy(teams_df)
        
        # Select data from 2018 only
        mlb_df_cleaned = mlb_df_cleaned[mlb_df_cleaned['year']==2018]
        
         # Get metropolitan area per team
        def get_metro_area(team):
            metropolitan_areas = {'New York City': ['New York Yankees', 'New York Mets'],
                                  'Los Angeles': ['Los Angeles Dodgers', 'Los Angeles Angels'],
                                  'San Francisco Bay Area': ['San Francisco Giants', 'Oakland Athletics'],
                                  'Chicago': ['Chicago Cubs', 'Chicago White Sox'],
                                  'Dallas–Fort Worth': ['Texas Rangers'],
                                  'Washington, D.C.': ['Washington Nationals'],
                                  'Philadelphia': ['Philadelphia Phillies'],
                                  'Boston': ['Boston Red Sox'],
                                  'Minneapolis–Saint Paul': ['Minnesota Twins'],
                                  'Denver': ['Colorado Rockies'],
                                  'Miami–Fort Lauderdale': ['Miami Marlins'],
                                  'Phoenix': ['Arizona Diamondbacks'],
                                  'Detroit': ['Detroit Tigers'],
                                  'Toronto': ['Toronto Blue Jays'],
                                  'Houston': ['Houston Astros'],
                                  'Atlanta': ['Atlanta Braves'],
                                  'Tampa Bay Area': ['Tampa Bay Rays'],
                                  'Pittsburgh': ['Pittsburgh Pirates'],
                                  'Cleveland': ['Cleveland Indians'],
                                  'Seattle': ['Seattle Mariners'],
                                  'Cincinnati': ['Cincinnati Reds'],
                                  'Kansas City': ['Kansas City Royals'],
                                  'St. Louis': ['St. Louis Cardinals'],
                                  'Baltimore': ['Baltimore Orioles'],
                                  'Milwaukee': ['Milwaukee Brewers'],
                                  'San Diego': ['San Diego Padres']}
            
            for m_area in metropolitan_areas:
                if team in metropolitan_areas[m_area]:
                    return m_area
        
        mlb_df_cleaned['Metropolitan area'] = mlb_df_cleaned['team'].apply(lambda x: get_metro_area(x))
        
        mlb_df_cleaned = mlb_df_cleaned.astype({'W': 'int64', 'L': 'int64'})
        
        return mlb_df_cleaned
    
    # Adjust cities df and set Metro Areas as index
    def clean_cities_df(cities_df):

        cities_clean = copy.copy(cities_df)

        # Rename Population column
        cities_clean.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

        # Change dtype for Population
        cities_clean = cities_clean.astype({'Population': 'int64'})

        cities_clean.set_index('Metropolitan area', inplace=True)

        return cities_clean['Population']
    
    # Clean mlb df
    teams_mlb_df = clean_mlb_df(mlb_df)
    
    # Group by Metropolitan area
    mlb_groups = teams_mlb_df.groupby('Metropolitan area').agg({'W': np.nansum, 'L': np.nansum})
    mlb_groups['Win/Loss ratio'] = mlb_groups['W']/(mlb_groups['W'] + mlb_groups['L'])
    
    # Adjust cities df and set Metro Areas as index
    metro_cities = clean_cities_df(cities)
    
    # Join mlb and cities dfs
    resulting_df = pd.merge(left=metro_cities, right=mlb_groups, how='inner', left_index=True, right_index=True)
    
    # raise NotImplementedError()
    
    population_by_region = resulting_df['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = resulting_df['Win/Loss ratio'].to_list() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import copy

def nfl_correlation():
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def clean_nfl_df(teams_df):
        
        nfl_df_cleaned = copy.copy(teams_df)
        
        # Select data from 2018 only
        nfl_df_cleaned = nfl_df_cleaned[nfl_df_cleaned['year']==2018]
        
        nfl_df_cleaned.drop(labels=[0, 5, 10, 15, 20, 25, 30, 35], inplace=True)
        
        # Eliminate * at the end of the name
        def fix_team_name(row):
            
            pattern = '.+?\*'
            if bool(re.search(pattern, row['team'])):
                row['team'] = re.sub('\*', '', row['team']).strip()
            
            pattern = '.+?\+'
            if bool(re.search(pattern, row['team'])):
                row['team'] = re.sub('\+', '', row['team']).strip()
            
            return row
        
        nfl_df_cleaned = nfl_df_cleaned.apply(lambda x: fix_team_name(x), axis='columns')
        
        # Get metropolitan area per team
        def get_metro_area(team):
            metropolitan_areas = {'New York City': ['New York Giants', 'New York Jets'],
                                  'Los Angeles': ['Los Angeles Rams', 'Los Angeles Chargers'],
                                  'San Francisco Bay Area': ['San Francisco 49ers', 'Oakland Raiders'],
                                  'Chicago': ['Chicago Bears'],
                                  'Dallas–Fort Worth': ['Dallas Cowboys'],
                                  'Washington, D.C.': ['Washington Redskins'],
                                  'Philadelphia': ['Philadelphia Eagles'],
                                  'Boston': ['New England Patriots'],
                                  'Minneapolis–Saint Paul': ['Minnesota Vikings'],
                                  'Denver': ['Denver Broncos'],
                                  'Miami–Fort Lauderdale': ['Miami Dolphins'],
                                  'Phoenix': ['Arizona Cardinals'],
                                  'Detroit': ['Detroit Lions'],
                                  'Houston': ['Houston Texans'],
                                  'Atlanta': ['Atlanta Falcons'],
                                  'Tampa Bay Area': ['Tampa Bay Buccaneers'],
                                  'Pittsburgh': ['Pittsburgh Steelers'],
                                  'Cleveland': ['Cleveland Browns'],
                                  'Seattle': ['Seattle Seahawks'],
                                  'Cincinnati': ['Cincinnati Bengals'],
                                  'Kansas City': ['Kansas City Chiefs'],
                                  'Baltimore': ['Baltimore Ravens'],
                                  'Charlotte': ['Carolina Panthers'],
                                  'Indianapolis': ['Indianapolis Colts'],
                                  'Nashville': ['Tennessee Titans'],
                                  'New Orleans': ['New Orleans Saints'],
                                  'Buffalo': ['Buffalo Bills'],
                                  'Jacksonville': ['Jacksonville Jaguars'],
                                  'Green Bay': ['Green Bay Packers']}
            
            for m_area in metropolitan_areas:
                if team in metropolitan_areas[m_area]:
                    return m_area
        
        nfl_df_cleaned['Metropolitan area'] = nfl_df_cleaned['team'].apply(lambda x: get_metro_area(x))
        
        nfl_df_cleaned = nfl_df_cleaned.astype({'W': 'int64', 'L': 'int64'})
        
        return nfl_df_cleaned
    
    # Adjust cities df and set Metro Areas as index
    def clean_cities_df(cities_df):

        cities_clean = copy.copy(cities_df)

        # Rename Population column
        cities_clean.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

        # Change dtype for Population
        cities_clean = cities_clean.astype({'Population': 'int64'})

        cities_clean.set_index('Metropolitan area', inplace=True)

        return cities_clean['Population']
    
    # Clean nfl df
    teams_nfl_df = clean_nfl_df(nfl_df)
    
    # Group by Metropolitan area
    nfl_groups = teams_nfl_df.groupby('Metropolitan area').agg({'W': np.nansum, 'L': np.nansum})
    nfl_groups['Win/Loss ratio'] = nfl_groups['W']/(nfl_groups['W'] + nfl_groups['L'])
    
    # Adjust cities df and set Metro Areas as index
    metro_cities = clean_cities_df(cities)
    
    # Join nfl and cities df
    resulting_df = pd.merge(left=metro_cities, right=nfl_groups, how='inner', left_index=True, right_index=True)
        
    # raise NotImplementedError()
    
    population_by_region = resulting_df['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = resulting_df['Win/Loss ratio'].to_list() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values